In [1]:
# default_exp models.nbeats

In [2]:
#hide
%load_ext autoreload
%autoreload 2

In [3]:
#export
import os
import time
import numpy as np
import pandas as pd
import random
from collections import defaultdict

import torch as t
from torch import optim
from pathlib import Path

from nixtla.models.nbeats_model import NBeats, NBeatsBlock, IdentityBasis, TrendBasis, SeasonalityBasis, ExogenousBasisGenericA, ExogenousBasisInterpretable
from nixtla.losses.pytorch import MAPELoss, MASELoss, SMAPELoss, MSELoss, MAELoss

In [4]:
#export
class Nbeats(object):
    """
    Future documentation
    """
    SEASONALITY_BLOCK = 'seasonality'
    TREND_BLOCK = 'trend'
    IDENTITY_BLOCK = 'identity'

    def __init__(self,
                 input_size_multiplier=2,
                 output_size=1,
                 shared_weights=True,
                 stack_types=[TREND_BLOCK, SEASONALITY_BLOCK],
                 n_blocks=[3,3],
                 n_layers=[4,4],
                 n_hidden=[256, 2048],
                 n_harmonics=1,
                 n_polynomials=2,
                 exogenous_n_channels=1,
                 exogenous_in_mlp=False,
                 batch_normalization=False,
                 dropout=0,
                 x_s_n_hidden=1,
                 learning_rate=0.001,
                 lr_decay=1.0,
                 n_lr_decay_steps=3,
                 weight_decay = 0,
                 n_iterations=300,
                 early_stopping = None,
                 loss='MAPE',
                 frequency=None,
                 seasonality=1,
                 random_seed=1,
                 device=None):
        super(Nbeats, self).__init__()

        self.input_size = int(input_size_multiplier*output_size)
        self.output_size = output_size
        self.shared_weights = shared_weights
        self.stack_types = stack_types
        self.n_blocks = n_blocks
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_harmonics = n_harmonics
        self.n_polynomials = n_polynomials
        self.exogenous_n_channels = exogenous_n_channels
        self.exogenous_in_mlp = exogenous_in_mlp
        self.batch_normalization = batch_normalization
        self.dropout = dropout
        self.x_s_n_hidden = x_s_n_hidden
        self.learning_rate = learning_rate
        self.lr_decay = lr_decay
        self.n_lr_decay_steps = n_lr_decay_steps
        self.weight_decay = weight_decay
        self.n_iterations = n_iterations
        self.early_stopping = early_stopping
        self.loss = loss
        self.frequency = frequency
        self.seasonality = seasonality
        self.random_seed = random_seed
        if device is None:
            device = 'cuda' if t.cuda.is_available() else 'cpu'
        self.device = device
        self.l1_lambda = 1e-3

        self._is_instantiated = False

    def create_stack(self):
        #print(f'| N-Beats')
        if self.exogenous_in_mlp:
            x_t_n_inputs = self.input_size + self.output_size * self.n_x_t
        else:
            x_t_n_inputs = self.input_size # y_lags
        block_list = []
        self.blocks_regularizer = []
        for i in range(len(self.stack_types)):
            #print(f'| --  Stack {self.stack_types[i]} (#{i})')
            for block_id in range(self.n_blocks[i]):
                self.blocks_regularizer += [0]
                if self.shared_weights and block_id>0:
                    nbeats_block = block_list[-1]
                else:
                    if self.stack_types[i] == 'seasonality':
                        nbeats_block = NBeatsBlock(x_t_n_inputs = x_t_n_inputs,
                                                   x_s_n_inputs = self.n_x_s,
                                                   x_s_n_hidden= self.x_s_n_hidden,
                                                   theta_dim=4 * int(
                                                        np.ceil(self.n_harmonics / 2 * self.output_size) - (self.n_harmonics - 1)),
                                                   basis=SeasonalityBasis(harmonics=self.n_harmonics,
                                                                                backcast_size=self.input_size,
                                                                                forecast_size=self.output_size),
                                                   n_layers=self.n_layers[i],
                                                   theta_n_hidden=self.n_hidden[i],
                                                   exogenous_in_mlp=self.exogenous_in_mlp,
                                                   batch_normalization=self.batch_normalization,
                                                   dropout=self.dropout)
                    elif self.stack_types[i] == 'trend':
                        nbeats_block = NBeatsBlock(x_t_n_inputs = x_t_n_inputs,
                                                   x_s_n_inputs = self.n_x_s,
                                                   x_s_n_hidden= self.x_s_n_hidden,
                                                   theta_dim=2 * (self.n_polynomials + 1),
                                                   basis=TrendBasis(degree_of_polynomial=self.n_polynomials,
                                                                            backcast_size=self.input_size,
                                                                            forecast_size=self.output_size),
                                                   n_layers=self.n_layers[i],
                                                   theta_n_hidden=self.n_hidden[i],
                                                   exogenous_in_mlp=self.exogenous_in_mlp,
                                                   batch_normalization=self.batch_normalization,
                                                   dropout=self.dropout)
                    elif self.stack_types[i] == 'identity':
                        nbeats_block = NBeatsBlock(x_t_n_inputs = x_t_n_inputs,
                                                   x_s_n_inputs = self.n_x_s,
                                                   x_s_n_hidden= self.x_s_n_hidden,
                                                   theta_dim=self.input_size + self.output_size,
                                                   basis=IdentityBasis(backcast_size=self.input_size,
                                                                       forecast_size=self.output_size),
                                                   n_layers=self.n_layers[i],
                                                   theta_n_hidden=self.n_hidden[i],
                                                   exogenous_in_mlp=self.exogenous_in_mlp,
                                                   batch_normalization=self.batch_normalization,
                                                   dropout=self.dropout)
                    elif self.stack_types[i] == 'exogenous':
                        nbeats_block = NBeatsBlock(x_t_n_inputs = x_t_n_inputs,
                                                   x_s_n_inputs = self.n_x_s,
                                                   x_s_n_hidden= self.x_s_n_hidden,
                                                   theta_dim=2*self.n_x_t,
                                                   basis=ExogenousBasisInterpretable(),
                                                   n_layers=self.n_layers[i],
                                                   theta_n_hidden=self.n_hidden[i],
                                                   exogenous_in_mlp=self.exogenous_in_mlp,
                                                   batch_normalization=self.batch_normalization,
                                                   dropout=self.dropout)
                    elif self.stack_types[i] == 'exogenous_g_a':
                        nbeats_block = NBeatsBlock(x_t_n_inputs = x_t_n_inputs,
                                                   x_s_n_inputs = self.n_x_s,
                                                   x_s_n_hidden= self.x_s_n_hidden,
                                                   theta_dim=2*(self.exogenous_n_channels),
                                                   basis=ExogenousBasisGenericA(self.exogenous_n_channels, self.n_x_t),
                                                   n_layers=self.n_layers[i],
                                                   theta_n_hidden=self.n_hidden[i],
                                                   exogenous_in_mlp=self.exogenous_in_mlp,
                                                   batch_normalization=self.batch_normalization,
                                                   dropout=self.dropout)
                        self.blocks_regularizer[-1] = 1                                                
                #print(f'     | -- {nbeats_block}')
                block_list.append(nbeats_block)
        return block_list

    def __loss_fn(self, loss_name: str):
        def loss(x, freq, forecast, target, mask):
            if loss_name == 'MAPE':
                return MAPELoss(y=target, y_hat=forecast, mask=mask) + self.l1_regularization()
            elif loss_name == 'MASE':
                return MASELoss(y=target, y_hat=forecast, y_insample=x, seasonality=freq, mask=mask) + self.l1_regularization()
            elif loss_name == 'SMAPE':
                return SMAPELoss(y=target, y_hat=forecast, mask=mask) + self.l1_regularization()
            elif loss_name == 'MSE':
                return MSELoss(y=target, y_hat=forecast, mask=mask) + self.l1_regularization()
            elif loss_name == 'MAE':
                return MAELoss(y=target, y_hat=forecast, mask=mask) + self.l1_regularization()
            else:
                raise Exception(f'Unknown loss function: {loss_name}')
        return loss
    
    def l1_regularization(self):
        l1_loss = 0
        for i, indicator in enumerate(self.blocks_regularizer):
            if indicator:
                l1_loss +=  self.l1_lambda*t.sum(t.abs(self.model.blocks[i].basis.weight))
        return l1_loss

    def to_tensor(self, x: np.ndarray) -> t.Tensor:
        tensor = t.as_tensor(x, dtype=t.float32).to(self.device)
        return tensor

    def evaluate_performance(self, ts_loader):
        #TODO: mas opciones que mae
        self.model.eval()

        losses = []
        with t.no_grad():
            for batch in iter(ts_loader):
                insample_y = self.to_tensor(batch['insample_y'])
                insample_x_t = self.to_tensor(batch['insample_x_t'])
                insample_mask = self.to_tensor(batch['insample_mask'])
                outsample_x_t = self.to_tensor(batch['outsample_x_t'])
                outsample_y = self.to_tensor(batch['outsample_y'])
                outsample_mask = self.to_tensor(batch['outsample_mask'])
                static_data = self.to_tensor(batch['static_data'])

                forecast = self.model(insample_y, insample_x_t, insample_mask, outsample_x_t, static_data)
                batch_loss = mae(y=forecast.cpu().data.numpy(),
                                 y_hat=outsample_y.cpu().data.numpy(),
                                 weights=outsample_mask.cpu().data.numpy())
                losses.append(batch_loss)
                break #TODO: remove this in future
        loss = np.mean(losses)
        return loss

    def fit(self, train_ts_loader, val_ts_loader=None, n_iterations=None, verbose=True, eval_steps=1):
        # TODO: Indexes hardcoded, information duplicated in train and val datasets

        # Random Seeds (model initialization)
        t.manual_seed(self.random_seed)
        np.random.seed(self.random_seed)
        random.seed(self.random_seed)

        # Attributes of ts_dataset
        self.n_x_t, self.n_x_s = train_ts_loader.get_n_variables()

        # Instantiate model
        if not self._is_instantiated:
            block_list = self.create_stack()
            self.model = NBeats(t.nn.ModuleList(block_list)).to(self.device)
            self._is_instantiated = True

        # Overwrite n_iterations and train datasets
        if n_iterations is None:
            n_iterations = self.n_iterations
        
        train_dataloader = iter(train_ts_loader)

        lr_decay_steps = n_iterations // self.n_lr_decay_steps
        if lr_decay_steps == 0:
            lr_decay_steps = 1

        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_decay_steps, gamma=self.lr_decay)
        training_loss_fn = self.__loss_fn(self.loss)

        if verbose and (n_iterations > 0):
            print('='*30+' Start fitting '+'='*30)
            print(f'Number of exogenous variables: {self.n_x_t}')
            print(f'Number of static variables: {self.n_x_s} , with dim_hidden: {self.x_s_n_hidden}')
            print(f'Number of iterations: {n_iterations}')
            print(f'Number of blocks: {len(self.model.blocks)}')

        #self.loss_dict = {} # Restart self.loss_dict
        start = time.time()
        self.trajectories = {'step':[],'train_loss':[], 'val_loss':[]}
        self.final_insample_loss = None
        self.final_outsample_loss = None

        # Training Loop
        best_val_loss = np.inf
        for step in range(n_iterations):
            self.model.train()
            train_ts_loader.train()

            batch = next(train_dataloader)

            insample_y = self.to_tensor(batch['insample_y'])
            insample_x_t = self.to_tensor(batch['insample_x_t'])
            insample_mask = self.to_tensor(batch['insample_mask'])

            outsample_x_t = self.to_tensor(batch['outsample_x_t'])
            outsample_y = self.to_tensor(batch['outsample_y'])
            outsample_mask = self.to_tensor(batch['outsample_mask'])

            static_data = self.to_tensor(batch['static_data'])

            optimizer.zero_grad()
            forecast = self.model(insample_y, insample_x_t, insample_mask, outsample_x_t, static_data)

            training_loss = training_loss_fn(x=insample_y, freq=self.seasonality, forecast=forecast,
                                            target=outsample_y, mask=outsample_mask)

            if np.isnan(float(training_loss)):
                break

            training_loss.backward()
            t.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            optimizer.step()

            lr_scheduler.step()
            if (step % eval_steps == 0):
                string = 'Step: {}, Time: {:03.3f}, Insample {}: {:.5f}'.format(step,
                                                                                time.time()-start,
                                                                                self.loss,
                                                                                training_loss.cpu().data.numpy())
                self.trajectories['step'].append(step)
                self.trajectories['train_loss'].append(training_loss.cpu().data.numpy())

                if val_ts_loader is not None:
                    loss = self.evaluate_performance(val_ts_loader)
                    string += ", Outsample {}: {:.5f}".format(self.loss, loss)
                    self.trajectories['val_loss'].append(loss)

                    if self.early_stopping:
                        if loss < best_val_loss:
                            counter = 0
                            best_val_loss = loss
                        else:
                            counter += 1
                        if counter >= self.early_stopping:
                            print(10*'-',' Stopped training by early stopping', 10*'-')
                            break
                
                print(string)

                self.model.train()
                train_ts_loader.train()

        #End of fitting
        if n_iterations >0:
            self.final_insample_loss = training_loss.cpu().data.numpy() #TODO: this is batch!
            string = 'Step: {}, Time: {:03.3f}, Insample {}: {:.5f}'.format(step,
                                                                            time.time()-start,
                                                                            self.loss,
                                                                            self.final_insample_loss)
            if val_ts_loader is not None:
                self.final_outsample_loss = self.evaluate_performance(val_ts_loader)
                string += ", Outsample {}: {:.5f}".format(self.loss, self.final_outsample_loss)
            print(string)
            print('='*30+'End fitting '+'='*30)

    def predict(self, ts_loader, X_test=None, eval_mode=False):

        ts_loader.eval()
        frequency = ts_loader.get_frequency()

        # Build forecasts
        unique_ids = ts_loader.get_meta_data_var('unique_id')
        last_ds = ts_loader.get_meta_data_var('last_ds') #TODO: ajustar of offset

        batch = next(iter(ts_loader))
        insample_y = self.to_tensor(batch['insample_y'])
        insample_x_t = self.to_tensor(batch['insample_x_t'])
        insample_mask = self.to_tensor(batch['insample_mask'])
        outsample_x_t = self.to_tensor(batch['outsample_x_t'])
        outsample_y = self.to_tensor(batch['outsample_y'])
        outsample_mask = self.to_tensor(batch['outsample_mask'])
        static_data = self.to_tensor(batch['static_data'])

        self.model.eval()
        with t.no_grad():
            forecast = self.model(insample_y, insample_x_t, insample_mask, outsample_x_t, static_data)

        if eval_mode:
            return forecast, outsample_y, outsample_mask

        # Predictions for panel
        Y_hat_panel = pd.DataFrame(columns=['unique_id', 'ds'])
        for i, unique_id in enumerate(unique_ids):
            Y_hat_id = pd.DataFrame([unique_id]*self.output_size, columns=["unique_id"])
            ds = pd.date_range(start=last_ds[i], periods=self.output_size+1, freq=self.frequency)
            Y_hat_id["ds"] = ds[1:]
            Y_hat_panel = Y_hat_panel.append(Y_hat_id, sort=False).reset_index(drop=True)

        forecast = forecast.cpu().detach().numpy()
        Y_hat_panel['y_hat'] = forecast.flatten()

        if X_test is not None:
            Y_hat_panel = X_test.merge(Y_hat_panel, on=['unique_id', 'ds'], how='left')
        
        return Y_hat_panel


    def save(self, model_dir, model_id):
    
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
    
        model_file = os.path.join(model_dir, f"model_{model_id}.model")
        print('Saving model to:\n {}'.format(model_file)+'\n')
        t.save({'model_state_dict': self.model.state_dict()}, model_file)

    def load(self, model_dir, model_id):

        model_file = os.path.join(model_dir, f"model_{model_id}.model")
        path = Path(model_file)

        assert path.is_file(), 'No model_*.model file found in this path!'

        print('Loading model from:\n {}'.format(model_file)+'\n')

        checkpoint = t.load(model_file, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.to(self.device)